# Import modules

In [1]:
from os.path import abspath, join, pardir
from nilearn import datasets
from roi_mean_cort import roi_mean_interface
from bids.layout import BIDSLayout
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.io import DataSink, DataGrabber
from nipype.algorithms.confounds import TSNR
from nipype.interfaces.utility import Function, IdentityInterface
from nilearn.input_data import NiftiLabelsMasker
from nilearn import plotting
from nipype.interfaces import fsl
from nipype.interfaces.utility import Rename

# Define variables for data and atlas

In [2]:
dataset_cort = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')
dataset_maps = dataset_cort.maps
project_path = abspath("data")

layout = BIDSLayout(project_path)

## Show path to output directory

In [3]:
 abspath(join(project_path, pardir,"output"))

'/home/jovyan/work/tSNR_ROI/output'

### Fetch files by subject_id and run_id
        subject_id and run_id are the plain ids as you get them from `BIDSlayout.get_subjects()`
        
    Parameters
    ---------
    layout: BIDSLayout
        layout to use
    subject_id: str
        SubjectID without sub-
    run_id: str
        RunID without run-
        
    Returns
    -------
    preproc,preprocessed_anat, brainmask, subject_id, run_id

### Def function get_files

In [4]:
def get_files(layout, subject_id, run_id):
    brainmask = layout.get(type="brainmask", return_type="file", subject=subject_id, run=run_id)[0]
    preproc = layout.get(type="preproc", return_type="file", subject=subject_id, run=run_id)
    all_preprocs = layout.get(subject=subject_id, type="preproc", return_type="file", modality="anat")
    print(brainmask)
    print(preproc)
    
    
    return preproc, brainmask, subject_id, run_id

### function resample atlas

In [5]:
def resample_atlas(source_img, template_img):
    from nilearn.image import resample_to_img
    from nilearn import datasets
    dataset_cort = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
    source_img = dataset_cort.maps
    resampled_atlas = resample_to_img(source_img, template_img, interpolation='nearest')
    return resampled_atlas

### function save atlas

In [6]:
def save_atlas(resampled_atlas):
    import os
    from os.path import abspath, join, pardir
    import numpy as np
    import nibabel as nib
    from nilearn.image import load_img
    from os.path import abspath
    work_dir = abspath(join("output"))
    atlas = load_img(resampled_atlas)
    save_atlas = atlas.to_filename(abspath('resampled_atlas.nii.gz'))
    saved_atlas = abspath('resampled_atlas.nii.gz')
    return saved_atlas

# Get Data

### Setup Node identitysource

In [7]:
identitysource = Node(IdentityInterface(fields=["subject_id", "run_id"]), name="identitysource")

identitysource.iterables= [('subject_id', layout.get_subjects()),
                           ('run_id', layout.get_runs())]

### Setup Node BIDSDataGrabber

In [8]:
BIDSDataGrabber = Node(Function(function=get_files,  input_names=["layout", "subject_id", "run_id",],
                                       output_names=["preproc", "brainmask", "subject_id", "run_id"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.layout = layout

# Processing Nodes

### setup Node for tsnr map calculation

In [9]:
tsnr = Node(TSNR(regress_poly=2), name='tsnr' )

### setup node for atlas resampling

In [10]:
resample_atlas = Node(name='resample_atlas',
               interface=Function(input_names=['template_img', 'source_img'],
                                  output_names=['resampled_atlas'],
                                  function=resample_atlas))
resample_atlas.inputs.source_img = dataset_cort.maps

### setup node for saving resampled atlas (Nifti1image to nii.gz)

In [11]:
save_atlas = Node(name='save_atlas',
               interface=Function(input_names=['resampled_atlas'],
                                  output_names=['saved_atlas'],
                                  function=save_atlas))

### Setup Node ApplyMask

In [12]:
apply_mask = Node(fsl.ApplyMask(), name="apply_mask")

### Set Atlas for mean calculation

In [13]:
roi_mean = Node(roi_mean_interface, name="roi_mean")

### Setup Node roi_mean
#### calculate mean values using function from roi_mean.py

### rename roi_mean.tsv to sub(subject_id)_run(run_id).tsv 

In [14]:
rename = Node(Rename(format_string="sub%(subject_id)s_run%(run_id)02d.tsv"), name='rename')

### Setup Node datasink

In [15]:
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = abspath(join(project_path, pardir,"output"))

### Connect Nodes, Setup Workflow

In [16]:
tsnr_wf = Workflow(name="TSNR_calculation")
tsnr_wf.base_dir = abspath('work_dir_cortical')
tsnr_wf.connect([(identitysource, BIDSDataGrabber, [('subject_id','subject_id'),
                                                    ('run_id','run_id')]),  
                 (BIDSDataGrabber, resample_atlas, [('preproc', 'template_img')]),
                 (BIDSDataGrabber, tsnr, [('preproc', 'in_file')]),
                 (BIDSDataGrabber, apply_mask, [('brainmask', 'mask_file')]),
                 (resample_atlas, save_atlas, [('resampled_atlas', 'resampled_atlas')]),
                 (save_atlas, apply_mask, [('saved_atlas', 'in_file')]),
                 (apply_mask, roi_mean, [('out_file', 'roi_file')]),
                 
                 (tsnr, roi_mean, [('tsnr_file', 'data_file')]),
                 (roi_mean, rename, [('out_file', 'in_file')]),
                 (BIDSDataGrabber, rename, [('subject_id','subject_id'),
                                                    ('run_id','run_id')]),
                 (tsnr, datasink, [('tsnr_file', 'cortical.@tsnr_map')]),
                 (save_atlas, datasink, [('saved_atlas', 'cortical.@resampled_atlas')]),
                 (apply_mask, datasink, [('out_file', 'cortical.@masked_atlas')]),
                 (rename, datasink, [('out_file', 'cortical.@mean_roi')])]
         )
         
tsnr_wf.write_graph(graph2use='colored', format='svg', simple_form=True)

181020-18:20:52,804 nipype.workflow INFO:
	 Generated workflow graph: /home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/graph.svg (graph2use=colored, simple_form=True).


'/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/graph.svg'

### Run the workflow

In [ ]:
tsnr_wf.run()

181020-18:20:53,828 nipype.workflow INFO:
	 Workflow TSNR_calculation settings: ['check', 'execution', 'logging', 'monitoring']
181020-18:24:53,442 nipype.workflow INFO:
	 Running serially.
181020-18:24:53,470 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.BIDSDataGrabber" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_11_subject_id_20/BIDSDataGrabber".
181020-18:24:57,132 nipype.workflow INFO:
	 [Node] Cached "TSNR_calculation.BIDSDataGrabber" - collecting precomputed outputs
181020-18:24:57,135 nipype.workflow INFO:
	 [Node] "TSNR_calculation.BIDSDataGrabber" found cached.
181020-18:24:57,136 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.tsnr" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_11_subject_id_20/tsnr".
181020-18:24:57,308 nipype.workflow INFO:
	 [Node] Cached "TSNR_calculation.tsnr" - collecting precomputed outputs
181020-18:24:57,309 nipype.workflow INFO:
	 [Node] "TSNR_calculation.tsnr" foun

181020-18:25:14,226 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.rename" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_10_subject_id_20/rename".
181020-18:25:14,580 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.rename".
181020-18:25:14,589 nipype.workflow INFO:
	 [Node] Running "rename" ("nipype.interfaces.utility.base.Rename")
181020-18:25:14,600 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.rename".
181020-18:25:14,602 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.datasink" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_10_subject_id_20/datasink".
181020-18:25:14,626 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.datasink".
181020-18:25:14,637 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
181020-18:25:14,665 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.datasink".
181020-18:25:14,666 ni

181020-18:25:36,70 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.roi_mean" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_8_subject_id_20/roi_mean".
181020-18:25:36,113 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.roi_mean".
181020-18:25:36,123 nipype.workflow INFO:
	 [Node] Running "roi_mean" ("nipype.interfaces.utility.wrappers.Function")
Calculationg Means
/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_8_subject_id_20/tsnr/tsnr.nii.gz
/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_8_subject_id_20/apply_mask/resampled_atlas_masked.nii.gz
[[1, 58.89224805450921, 30.286636995634421], [32, 54.116188174978788, 22.889749479428207], [28, 59.011437884647947, 25.97013012743593]]
181020-18:25:36,212 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.roi_mean".
181020-18:25:36,214 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.rename" in "/home/jovyan/work/tSN

181020-18:25:52,449 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.apply_mask".
181020-18:25:52,459 nipype.workflow INFO:
	 [Node] Running "apply_mask" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_6_subject_id_20/save_atlas/resampled_atlas.nii.gz -mas /home/jovyan/work/tSNR_ROI/data/derivatives/fmriprep/sub-20/func/sub-20_task-rest_run-06_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz /home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_6_subject_id_20/apply_mask/resampled_atlas_masked.nii.gz
181020-18:25:57,526 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.apply_mask".
181020-18:25:57,528 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.roi_mean" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_6_subject_id_20/roi_mean".
181020-18:25:57,551 nipype.workflow INFO:
	 [Node] Outdated 

181020-18:26:12,732 nipype.workflow INFO:
	 [Node] "TSNR_calculation.resample_atlas" found cached.
181020-18:26:12,734 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.save_atlas" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_4_subject_id_20/save_atlas".
181020-18:26:12,752 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.save_atlas".
181020-18:26:13,58 nipype.workflow INFO:
	 [Node] Running "save_atlas" ("nipype.interfaces.utility.wrappers.Function")
181020-18:26:13,235 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.save_atlas".
181020-18:26:13,236 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.apply_mask" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_4_subject_id_20/apply_mask".
181020-18:26:13,412 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.apply_mask".
181020-18:26:13,418 nipype.workflow INFO:
	 [Node] Running "apply_mask" ("nipype.interf

...

181020-19:39:48,99 nipype.workflow INFO:
	 [Node] Running "apply_mask" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_6_subject_id_01/save_atlas/resampled_atlas.nii.gz -mas /home/jovyan/work/tSNR_ROI/data/derivatives/fmriprep/sub-01/func/sub-01_task-rest_run-06_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz /home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_6_subject_id_01/apply_mask/resampled_atlas_masked.nii.gz
181020-19:39:56,126 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.apply_mask".
181020-19:39:56,128 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.roi_mean" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_6_subject_id_01/roi_mean".
181020-19:39:56,132 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.roi_mean".
181020-19:39:56,137 nipype.workflow INFO:
	 [Node] Running "roi

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


181020-19:39:56,623 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.rename".
181020-19:39:56,631 nipype.workflow INFO:
	 [Node] Running "rename" ("nipype.interfaces.utility.base.Rename")
181020-19:39:56,642 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.rename".
181020-19:39:56,644 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.datasink" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_6_subject_id_01/datasink".
181020-19:39:56,655 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.datasink".
181020-19:39:56,664 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
181020-19:39:56,705 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.datasink".
181020-19:39:56,706 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.BIDSDataGrabber" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_5_subject_id_01/BIDSDataGrabber".
181020

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


181020-19:40:09,724 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.rename".
181020-19:40:09,739 nipype.workflow INFO:
	 [Node] Running "rename" ("nipype.interfaces.utility.base.Rename")
181020-19:40:09,749 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.rename".
181020-19:40:09,750 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.datasink" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_5_subject_id_01/datasink".
181020-19:40:09,761 nipype.workflow INFO:
	 [Node] Outdated cache found for "TSNR_calculation.datasink".
181020-19:40:09,769 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
181020-19:40:09,792 nipype.workflow INFO:
	 [Node] Finished "TSNR_calculation.datasink".
181020-19:40:09,793 nipype.workflow INFO:
	 [Node] Setting-up "TSNR_calculation.BIDSDataGrabber" in "/home/jovyan/work/tSNR_ROI/work_dir_cortical/TSNR_calculation/_run_id_4_subject_id_01/BIDSDataGrabber".
181020